# 1.ライブラリの読み込み
上から順に
* グラフを書くためにmatplotlib
* データの整理のためにpandas
* 数学的な計算のためにnumpy
* 時系列データを扱うためにdatetime
* matplotlibでdatetimeのフォーマットを変えるためにmdates

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.dates as mdates
from scipy import signal

# 2.センサーデータの読みこみと確認
ファイルの読み込みに使用した変数名の命名則については以下の通りである．

例　セッション1のAさんのセンサデータ →　data1_d
　　セッション1のAさんのスケーリングデータ　→　data1_d_label_50

In [2]:
%time
#sensor_dataの読み込み

# data1_d = pd.read_csv('sensor_data/session1/TSND151/raw_data_time/acc_gyr/Conv-mem-A胸-20201111-121953632-ags.csv', encoding="shift-jis")
# data1_b = pd.read_csv('sensor_data/session1/TSND151/raw_data_time/acc_gyr/Conv-mem-B胸-20201111-122231707-ags.csv', encoding="shift-jis")
# data1_c = pd.read_csv('sensor_data/session1/TSND151/raw_data_time/acc_gyr/Conv-mem-C胸-20201111-121953885-ags.csv', encoding="shift-jis")
data1_d = pd.read_csv('sensor_data/session1/TSND151/raw_data_time/acc_gyr/Conv-mem-D胸-20201111-121954011-ags.csv', encoding="shift-jis")
# data2_d = pd.read_csv('sensor_data/session2/TSND151/raw_data_time/acc_gyr/Conv-mem-A胸-20201111-145759193-ags.csv', encoding="shift-jis")
# data2_b = pd.read_csv('sensor_data/session2/TSND151/raw_data_time/acc_gyr/Conv-mem-B胸-20201111-145733362-ags.csv', encoding="shift-jis")
# data2_c = pd.read_csv('sensor_data/session2/TSND151/raw_data_time/acc_gyr/Conv-mem-C胸-20201111-150018208-ags.csv', encoding="shift-jis")
# data2_d = pd.read_csv('sensor_data/session2/TSND151/raw_data_time/acc_gyr/Conv-mem-D胸-20201111-145806707-ags.csv', encoding="shift-jis")

#emotion_labelの読み込み

data1_d_label_50 = pd.read_csv('emotion_label/session1/D/emotion_D_20201111_1_50.csv')
data1_d_label_60 = pd.read_csv('emotion_label/session1/D/emotion_D_20201111_1_60.csv')
data1_d_label_70 = pd.read_csv('emotion_label/session1/D/emotion_D_20201111_1_70.csv')
data1_d_label_80 = pd.read_csv('emotion_label/session1/D/emotion_D_20201111_1_80.csv')

# data2_d_label_50 = pd.read_csv('emotion_label/D/emotion_D_20201111_2_50_es.csv')
# data2_d_label_60 = pd.read_csv('emotion_label/D/emotion_D_20201111_2_60_es.csv')
# data2_d_label_70 = pd.read_csv('emotion_label/D/emotion_D_20201111_2_70_es.csv')
# data2_d_label_80 = pd.read_csv('emotion_label/D/emotion_D_20201111_2_80_es.csv')

Wall time: 0 ns


## 2.1 TSND151加速度，角速度センサデータとスケーリングデータの中身
今回のセンサデータの列はセンサの種類，時間，加速度のx，y，z，角速度x，y，zで構成されており，加速度の単位は0.1mG，角速度の単位は0.01dpsである．

スケーリングデータの列は，レコーディング動画の経過時間，感情のレベル値，発話されたユーザ名となっている．emotion_levelの感情値については今回は以下のように考えた．

-4〜-1→negative  0→neutral  １〜4→positive

In [3]:
data1_d_label_50

,start_time,finish_time,emotion_level,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,6:50,6:54,4,NaN,NaN,NaN
1,6:51,6:55,4,NaN,NaN,NaN
2,6:52,6:56,4,NaN,NaN,NaN
3,6:53,6:57,4,NaN,NaN,NaN
4,6:54,6:58,4,NaN,NaN,NaN
...,...,...,...,...,...,...
636,19:19,19:23,-2,NaN,NaN,NaN
637,19:20,19:24,-2,NaN,NaN,NaN
638,19:21,19:25,-2,NaN,NaN,NaN
639,19:22,19:26,-2,NaN,NaN,NaN


In [4]:
#欠損値削除
def dropna(label):
    a = label.dropna(how='all', axis=1)
    return a

data1_d_label_50 = dropna(data1_d_label_50)
data1_d_label_60 = dropna(data1_d_label_60)
data1_d_label_70 = dropna(data1_d_label_70)
data1_d_label_80 = dropna(data1_d_label_80)

# data2_d_label_50 = dropna(data2_d_label_50)
# data2_d_label_60 = dropna(data2_d_label_60)
# data2_d_label_70 = dropna(data2_d_label_70)
# data2_d_label_80 = dropna(data2_d_label_80)

In [5]:
data1_d_label_50

,start_time,finish_time,emotion_level
0,6:50,6:54,4
1,6:51,6:55,4
2,6:52,6:56,4
3,6:53,6:57,4
4,6:54,6:58,4
...,...,...,...
636,19:19,19:23,-2
637,19:20,19:24,-2
638,19:21,19:25,-2
639,19:22,19:26,-2


In [6]:
#生データの確認
display(data1_d)
display(data1_d_label_50)

,種別,時刻,加速度X,加速度Y,加速度Z,角速度X,角速度Y,角速度Z
0,ags,12:19:54.065,9875,574,3327,-30,274,89
1,ags,12:19:54.070,9858,557,3332,-43,146,46
2,ags,12:19:54.075,9878,501,3312,-134,195,34
3,ags,12:19:54.080,9909,496,3273,-122,91,52
4,ags,12:19:54.085,9919,469,3170,-61,0,40
...,...,...,...,...,...,...,...,...
485707,ags,13:00:23.125,2956,723,10092,395,-761,1900
485708,ags,13:00:23.130,3137,875,10009,560,-408,1943
485709,ags,13:00:23.135,3266,953,9936,706,-280,1894
485710,ags,13:00:23.140,3371,1175,10199,755,-115,1833


,start_time,finish_time,emotion_level
0,6:50,6:54,4
1,6:51,6:55,4
2,6:52,6:56,4
3,6:53,6:57,4
4,6:54,6:58,4
...,...,...,...
636,19:19,19:23,-2
637,19:20,19:24,-2
638,19:21,19:25,-2
639,19:22,19:26,-2


In [7]:
#合成加速度と合成角速度の算出
def norm_order(sensor_data):
    sensor_data["合成加速度"] = np.sqrt(sensor_data["加速度X"]*sensor_data["加速度X"] + sensor_data["加速度Y"]*sensor_data["加速度Y"] + sensor_data["加速度Z"]*sensor_data["加速度Z"])
    sensor_data["合成角速度"] = np.sqrt(sensor_data["角速度X"]*sensor_data["角速度X"] + sensor_data["角速度Y"]*sensor_data["角速度Y"] + sensor_data["角速度Z"]*sensor_data["角速度Z"])
    
norm_order(data1_d)
# norm_order(data1_b)
# norm_order(data1_c)
# norm_order(data1_d)
# norm_order(data2_d)
# norm_order(data2_b)
# norm_order(data2_c)
# norm_order(data2_d)

In [8]:
data1_d

,種別,時刻,加速度X,加速度Y,加速度Z,角速度X,角速度Y,角速度Z,合成加速度,合成角速度
0,ags,12:19:54.065,9875,574,3327,-30,274,89,10436.188480,289.649789
1,ags,12:19:54.070,9858,557,3332,-43,146,46,10420.779098,159.000000
2,ags,12:19:54.075,9878,501,3312,-134,195,34,10430.495146,239.033470
3,ags,12:19:54.080,9909,496,3273,-122,91,52,10447.335833,160.838428
4,ags,12:19:54.085,9919,469,3170,-61,0,40,10423.791153,72.945185
...,...,...,...,...,...,...,...,...,...,...
485707,ags,13:00:23.125,2956,723,10092,395,-761,1900,10540.831514,2084.501379
485708,ags,13:00:23.130,3137,875,10009,560,-408,1943,10525.515427,2062.841002
485709,ags,13:00:23.135,3266,953,9936,706,-280,1894,10502.335978,2040.605792
485710,ags,13:00:23.140,3371,1175,10199,755,-115,1833,10805.733062,1985.733869


# 3.スケーリングデータに即したセンサデータの抽出
スケーリングデータに即したセンサデータの取得方法は以下の通りである．
1. センサデータ及びスケーリングデータの時刻列をdatetime化
2. timedelta関数を使用して，時刻計算を行い，実験実施日に合わせていく
3. センサデータの時刻列をインデックス化

  時刻列をインデックスにすることで，時間を使ってのデータのスライスが可能となる

4. スケーリングデータの時刻列を文字列型のリストにする
5. ４の操作を行うことでセンサデータからスケーリングデータに即したデータを抽出するのが容易になる

In [9]:
#センサデータの時刻合わせ
def datetime_order(sensor_data):
    sensor_data['時刻'] = pd.to_datetime(sensor_data['時刻'], format='%H:%M:%S.%f')
    sensor_data['時刻'] = sensor_data['時刻'] + dt.timedelta(days=44144)

datetime_order(data1_d)
# datetime_order(data1_b)
# datetime_order(data1_c)
# datetime_order(data1_d)

# datetime_order(data2_d)
# datetime_order(data2_b)
# datetime_order(data2_c)
# datetime_order(data2_d)

In [10]:
data1_d

,種別,時刻,加速度X,加速度Y,加速度Z,角速度X,角速度Y,角速度Z,合成加速度,合成角速度
0,ags,2020-11-11 12:19:54.065,9875,574,3327,-30,274,89,10436.188480,289.649789
1,ags,2020-11-11 12:19:54.070,9858,557,3332,-43,146,46,10420.779098,159.000000
2,ags,2020-11-11 12:19:54.075,9878,501,3312,-134,195,34,10430.495146,239.033470
3,ags,2020-11-11 12:19:54.080,9909,496,3273,-122,91,52,10447.335833,160.838428
4,ags,2020-11-11 12:19:54.085,9919,469,3170,-61,0,40,10423.791153,72.945185
...,...,...,...,...,...,...,...,...,...,...
485707,ags,2020-11-11 13:00:23.125,2956,723,10092,395,-761,1900,10540.831514,2084.501379
485708,ags,2020-11-11 13:00:23.130,3137,875,10009,560,-408,1943,10525.515427,2062.841002
485709,ags,2020-11-11 13:00:23.135,3266,953,9936,706,-280,1894,10502.335978,2040.605792
485710,ags,2020-11-11 13:00:23.140,3371,1175,10199,755,-115,1833,10805.733062,1985.733869


In [11]:
data1_d_label_50['start_time'] = pd.to_datetime(data1_d_label_50['start_time'], format='%M:%S')
data1_d_label_50['finish_time'] = pd.to_datetime(data1_d_label_50['finish_time'], format='%M:%S')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [12]:
data1_d_label_60['start_time'] = pd.to_datetime(data1_d_label_60['start_time'], format='%M:%S')
data1_d_label_60['finish_time'] = pd.to_datetime(data1_d_label_60['finish_time'], format='%M:%S')

In [13]:
data1_d_label_70['start_time'] = pd.to_datetime(data1_d_label_70['start_time'], format='%M:%S')
data1_d_label_70['finish_time'] = pd.to_datetime(data1_d_label_70['finish_time'], format='%M:%S')

In [14]:
data1_d_label_80['start_time'] = pd.to_datetime(data1_d_label_80['start_time'], format='%M:%S')
data1_d_label_80['finish_time'] = pd.to_datetime(data1_d_label_80['finish_time'], format='%M:%S')

In [15]:
# data2_d_label_50['start_time'] = pd.to_datetime(data2_d_label_50['start_time'], format='%M:%S')
# data2_d_label_50['finish_time'] = pd.to_datetime(data2_d_label_50['finish_time'], format='%M:%S')

In [16]:
# data2_d_label_60['start_time'] = pd.to_datetime(data2_d_label_60['start_time'], format='%M:%S')
# data2_d_label_60['finish_time'] = pd.to_datetime(data2_d_label_60['finish_time'], format='%M:%S')

In [17]:
# data2_d_label_70['start_time'] = pd.to_datetime(data2_d_label_70['start_time'], format='%M:%S')
# data2_d_label_70['finish_time'] = pd.to_datetime(data2_d_label_70['finish_time'], format='%M:%S')

In [18]:
# data2_d_label_80['start_time'] = pd.to_datetime(data2_d_label_80['start_time'], format='%M:%S')
# data2_d_label_80['finish_time'] = pd.to_datetime(data2_d_label_80['finish_time'], format='%M:%S')

In [19]:
#編集したデータの確認(確認のためセッション1のAさんのデータを表示している)
display(data1_d)
display(data1_d_label_50)

,種別,時刻,加速度X,加速度Y,加速度Z,角速度X,角速度Y,角速度Z,合成加速度,合成角速度
0,ags,2020-11-11 12:19:54.065,9875,574,3327,-30,274,89,10436.188480,289.649789
1,ags,2020-11-11 12:19:54.070,9858,557,3332,-43,146,46,10420.779098,159.000000
2,ags,2020-11-11 12:19:54.075,9878,501,3312,-134,195,34,10430.495146,239.033470
3,ags,2020-11-11 12:19:54.080,9909,496,3273,-122,91,52,10447.335833,160.838428
4,ags,2020-11-11 12:19:54.085,9919,469,3170,-61,0,40,10423.791153,72.945185
...,...,...,...,...,...,...,...,...,...,...
485707,ags,2020-11-11 13:00:23.125,2956,723,10092,395,-761,1900,10540.831514,2084.501379
485708,ags,2020-11-11 13:00:23.130,3137,875,10009,560,-408,1943,10525.515427,2062.841002
485709,ags,2020-11-11 13:00:23.135,3266,953,9936,706,-280,1894,10502.335978,2040.605792
485710,ags,2020-11-11 13:00:23.140,3371,1175,10199,755,-115,1833,10805.733062,1985.733869


,start_time,finish_time,emotion_level
0,1900-01-01 00:06:50,1900-01-01 00:06:54,4
1,1900-01-01 00:06:51,1900-01-01 00:06:55,4
2,1900-01-01 00:06:52,1900-01-01 00:06:56,4
3,1900-01-01 00:06:53,1900-01-01 00:06:57,4
4,1900-01-01 00:06:54,1900-01-01 00:06:58,4
...,...,...,...
636,1900-01-01 00:19:19,1900-01-01 00:19:23,-2
637,1900-01-01 00:19:20,1900-01-01 00:19:24,-2
638,1900-01-01 00:19:21,1900-01-01 00:19:25,-2
639,1900-01-01 00:19:22,1900-01-01 00:19:26,-2


### 補足(実験実施時間)

本実験は，zoomレコーディング動画の経過時間でスケーリングデータのemotion_levelをつけたため動画の開始時刻を使って実際の実験実施時間と合わせる必要がある．

方法としては，経過時間と動画の開始時刻を足して実験実施時間を算出する．

動画の開始時間↓

第1セッション12時34分30秒開始

第2セッション15時11分44秒開始

In [20]:
data1_d_label_50['start_time'] = data1_d_label_50['start_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)
data1_d_label_50['finish_time'] = data1_d_label_50['finish_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [21]:
data1_d_label_60['start_time'] = data1_d_label_60['start_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)
data1_d_label_60['finish_time'] = data1_d_label_60['finish_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)

In [22]:
data1_d_label_70['start_time'] = data1_d_label_70['start_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)
data1_d_label_70['finish_time'] = data1_d_label_70['finish_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)

In [23]:
data1_d_label_80['start_time'] = data1_d_label_80['start_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)
data1_d_label_80['finish_time'] = data1_d_label_80['finish_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)

In [24]:
# data2_d_label_50['start_time'] = data2_d_label_50['start_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)
# data2_d_label_50['finish_time'] = data2_d_label_50['finish_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)

In [25]:
# data2_d_label_60['start_time'] = data2_d_label_60['start_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)
# data2_d_label_60['finish_time'] = data2_d_label_60['finish_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)

In [26]:
# data2_d_label_70['start_time'] = data2_d_label_70['start_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)
# data2_d_label_70['finish_time'] = data2_d_label_70['finish_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)

In [27]:
# data2_d_label_80['start_time'] = data2_d_label_80['start_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)
# data2_d_label_80['finish_time'] = data2_d_label_80['finish_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)

In [28]:
#編集したデータの確認(確認のためセッション1のAさんのデータを表示している)
display(data1_d_label_80)

,start_time,finish_time,emotion_level
0,2020-11-11 12:41:24,2020-11-11 12:41:28,4
1,2020-11-11 12:41:25,2020-11-11 12:41:29,4
2,2020-11-11 12:41:26,2020-11-11 12:41:30,4
3,2020-11-11 12:41:27,2020-11-11 12:41:31,4
4,2020-11-11 12:41:28,2020-11-11 12:41:32,4
...,...,...,...
599,2020-11-11 12:53:49,2020-11-11 12:53:53,-2
600,2020-11-11 12:53:50,2020-11-11 12:53:54,-2
601,2020-11-11 12:53:51,2020-11-11 12:53:55,-2
602,2020-11-11 12:53:52,2020-11-11 12:53:56,-2


In [29]:
#datetime化したラベルデータを保存
# data1_d_label_50.to_csv('emotion_label/label_datetime/emotion_A_20201111_1_datetime.csv', index=False)
# data1_d_label_60.to_csv('emotion_label/label_datetime/emotion_B_20201111_1_datetime.csv', index=False)
# data1_d_label_70.to_csv('emotion_label/label_datetime/emotion_C_20201111_1_datetime.csv', index=False)
# data1_d_label_80.to_csv('emotion_label/label_datetime/emotion_D_20201111_1_datetime.csv', index=False)

# data2_d_label_50.to_csv('emotion_label/label_datetime/emotion_A_20201111_2_datetime.csv', index=False)
# data2_d_label_60.to_csv('emotion_label/label_datetime/emotion_B_20201111_2_datetime.csv', index=False)
# data2_d_label_70.to_csv('emotion_label/label_datetime/emotion_C_20201111_2_datetime.csv', index=False)
# data2_d_label_80.to_csv('emotion_label/label_datetime/emotion_D_20201111_2_datetime.csv', index=False)

In [30]:
#datetime化した加速度及び角速度データを保存
# data1_d.to_csv('sensor_data/session1/TSND151/raw_data_datetime/acc_gyr/Conv-mem-A胸-20201111-121953632-ags_datetime.csv', index=False)
# data1_b.to_csv('sensor_data/session1/TSND151/raw_data_datetime/acc_gyr/Conv-mem-B胸-20201111-122231707-ags_datetime.csv', index=False)
# data1_c.to_csv('sensor_data/session1/TSND151/raw_data_datetime/acc_gyr/Conv-mem-C胸-20201111-121953885-ags_datetime.csv', index=False)
# data1_d.to_csv('sensor_data/session1/TSND151/raw_data_datetime/acc_gyr/Conv-mem-D胸-20201111-121954011-ags_datetime.csv', index=False)

# data2_d.to_csv('sensor_data/session2/TSND151/raw_data_datetime/acc_gyr/Conv-mem-A胸-20201111-145759193-ags_datetime.csv', index=False)
# data2_b.to_csv('sensor_data/session2/TSND151/raw_data_datetime/acc_gyr/Conv-mem-B胸-20201111-145733362-ags_datetime.csv', index=False)
# data2_c.to_csv('sensor_data/session2/TSND151/raw_data_datetime/acc_gyr/Conv-mem-C胸-20201111-150018208-ags_datetime.csv', index=False)
# data2_d.to_csv('sensor_data/session2/TSND151/raw_data_datetime/acc_gyr/Conv-mem-D胸-20201111-145806707-ags_datetime.csv', index=False)

In [31]:
#時刻をインデックスに
data1_d.set_index("時刻", inplace=True)
# data1_b.set_index("時刻", inplace=True)
# data1_c.set_index("時刻", inplace=True)
# data1_d.set_index("時刻", inplace=True)

# data2_d.set_index("時刻", inplace=True)
# data2_b.set_index("時刻", inplace=True)
# data2_c.set_index("時刻", inplace=True)
# data2_d.set_index("時刻", inplace=True)

In [32]:
#編集したデータの確認(確認のためセッション1のAさんのデータを表示している)
display(data1_d)

,種別,加速度X,加速度Y,加速度Z,角速度X,角速度Y,角速度Z,合成加速度,合成角速度
時刻,,,,,,,,,
2020-11-11 12:19:54.065,ags,9875,574,3327,-30,274,89,10436.188480,289.649789
2020-11-11 12:19:54.070,ags,9858,557,3332,-43,146,46,10420.779098,159.000000
2020-11-11 12:19:54.075,ags,9878,501,3312,-134,195,34,10430.495146,239.033470
2020-11-11 12:19:54.080,ags,9909,496,3273,-122,91,52,10447.335833,160.838428
2020-11-11 12:19:54.085,ags,9919,469,3170,-61,0,40,10423.791153,72.945185
...,...,...,...,...,...,...,...,...,...
2020-11-11 13:00:23.125,ags,2956,723,10092,395,-761,1900,10540.831514,2084.501379
2020-11-11 13:00:23.130,ags,3137,875,10009,560,-408,1943,10525.515427,2062.841002
2020-11-11 13:00:23.135,ags,3266,953,9936,706,-280,1894,10502.335978,2040.605792


In [33]:
data1_d_label_50["start_time"] = data1_d_label_50["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
data1_d_label_50["finish_time"] = data1_d_label_50["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [34]:
data1_d_label_60["start_time"] = data1_d_label_60["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
data1_d_label_60["finish_time"] = data1_d_label_60["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [35]:
data1_d_label_70["start_time"] = data1_d_label_70["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
data1_d_label_70["finish_time"] = data1_d_label_70["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [36]:
data1_d_label_80["start_time"] = data1_d_label_80["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
data1_d_label_80["finish_time"] = data1_d_label_80["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [37]:
# data2_d_label_50["start_time"] = data2_d_label_50["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
# data2_d_label_50["finish_time"] = data2_d_label_50["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [38]:
# data2_d_label_50["start_time"] = data2_d_label_50["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
# data2_d_label_50["finish_time"] = data2_d_label_50["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [39]:
# data2_d_label_50["start_time"] = data2_d_label_50["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
# data2_d_label_50["finish_time"] = data2_d_label_50["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [40]:
# data2_d_label_50["start_time"] = data2_d_label_50["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
# data2_d_label_50["finish_time"] = data2_d_label_50["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [41]:
#編集したデータの確認(確認のためセッション1のAさんのデータを表示している)
display(data1_d_label_70)

,start_time,finish_time,emotion_level
0,2020-11-11 12:41:22,2020-11-11 12:41:26,4
1,2020-11-11 12:41:23,2020-11-11 12:41:27,4
2,2020-11-11 12:41:24,2020-11-11 12:41:28,4
3,2020-11-11 12:41:25,2020-11-11 12:41:29,4
4,2020-11-11 12:41:26,2020-11-11 12:41:30,4
...,...,...,...
613,2020-11-11 12:53:49,2020-11-11 12:53:53,-2
614,2020-11-11 12:53:50,2020-11-11 12:53:54,-2
615,2020-11-11 12:53:51,2020-11-11 12:53:55,-2
616,2020-11-11 12:53:52,2020-11-11 12:53:56,-2


In [42]:
#startとfinishをリスト化
data1_d_50_start_time = list(data1_d_label_50["start_time"])
data1_d_50_finish_time = list(data1_d_label_50["finish_time"])

data1_d_60_start_time = list(data1_d_label_60["start_time"])
data1_d_60_finish_time = list(data1_d_label_60["finish_time"])

data1_d_70_start_time = list(data1_d_label_70["start_time"])
data1_d_70_finish_time = list(data1_d_label_70["finish_time"])

data1_d_80_start_time = list(data1_d_label_80["start_time"])
data1_d_80_finish_time = list(data1_d_label_80["finish_time"])

# data2_d_50_start_time = list(data2_d_label_50["start_time"])
# data2_d_50_finish_time = list(data2_d_label_50["finish_time"])

# data2_d_60_start_time = list(data2_d_label_60["start_time"])
# data2_d_60_finish_time = list(data2_d_label_60["finish_time"])

# data2_d_70_start_time = list(data2_d_label_70["start_time"])
# data2_d_70_finish_time = list(data2_d_label_70["finish_time"])

# data2_d_80_start_time = list(data2_d_label_80["start_time"])
# data2_d_80_finish_time = list(data2_d_label_80["finish_time"])

In [43]:
def sensor_segmentation_order(data, data_start_time, data_finish_time, data_label):
    a = [data[data_start_time[i]:data_finish_time[i]]for i in range(0, len(data_label), 1)]
    return a

data1_d_record_1 = sensor_segmentation_order(data1_d, data1_d_50_start_time, data1_d_50_finish_time, data1_d_label_50)
data1_d_record_2 = sensor_segmentation_order(data1_d, data1_d_60_start_time, data1_d_60_finish_time, data1_d_label_60)
data1_d_record_3 = sensor_segmentation_order(data1_d, data1_d_70_start_time, data1_d_70_finish_time, data1_d_label_70)
data1_d_record_4 = sensor_segmentation_order(data1_d, data1_d_80_start_time, data1_d_80_finish_time, data1_d_label_80)

# data2_d_record_1 = sensor_segmentation_order(data2_d, data2_d_50_start_time, data2_d_50_finish_time, data2_d_label_50)
# data2_d_record_2 = sensor_segmentation_order(data2_d, data2_d_60_start_time, data2_d_60_finish_time, data2_d_label_60)
# data2_d_record_3 = sensor_segmentation_order(data2_d, data2_d_70_start_time, data2_d_70_finish_time, data2_d_label_70)
# data2_d_record_4 = sensor_segmentation_order(data2_d, data2_d_80_start_time, data2_d_80_finish_time, data2_d_label_80)

# 4.特徴量算出
感情の識別を行うため学習器作成のために特徴量の算出を行う．今回特徴量としたのは，加速度の各軸及び角速度の各軸における時間領域での平均値，分散値，最大値，最小値及び，周波数領域での特徴量である最大ピーク周波数，最大ピーク値である．プログラムの処理手順を以下に示す．

1. 平均値の算出
2. 分散値の算出
3. 最大値の算出
4. 最小値の算出
5. 最大ピーク周波数及び最大ピーク値の算出
5. 全特徴量の数値を表記したデータフレームを作成

## 4.1 平均値算出
### 4.1.1 加速度

In [44]:
#加速度x,y,z,norm平均値
def acc_gyr_mean_order(data_record, data_label):
    a = [np.mean(data_record[i]["加速度X"]) for i in range(0, len(data_label), 1)]
    b = [np.mean(data_record[i]["加速度Y"]) for i in range(0, len(data_label), 1)]
    c = [np.mean(data_record[i]["加速度Z"]) for i in range(0, len(data_label), 1)]
    d = [np.mean(data_record[i]["合成加速度"]) for i in range(0, len(data_label), 1)]
    a = pd.DataFrame(a)
    b = pd.DataFrame(b)
    c = pd.DataFrame(c)
    d = pd.DataFrame(d)
    acc_mean_list = pd.concat([a, b, c], axis=1, ignore_index=True)
    acc_mean_list.columns = ["accx_mean", "accy_mean", "accz_mean"]
#     acc_mean_list = pd.concat([a, b, c, d], axis=1, ignore_index=True)
#     acc_mean_list.columns = ["accx_mean", "accy_mean", "accz_mean", "accn_mean"]
    e = [np.mean(data_record[i]["角速度X"]) for i in range(0, len(data_label), 1)]
    f = [np.mean(data_record[i]["角速度Y"]) for i in range(0, len(data_label), 1)]
    g = [np.mean(data_record[i]["角速度Z"]) for i in range(0, len(data_label), 1)]
    h = [np.mean(data_record[i]["合成角速度"]) for i in range(0, len(data_label), 1)]
    e = pd.DataFrame(e)
    f = pd.DataFrame(f)
    g = pd.DataFrame(g)
    h = pd.DataFrame(h)
    gyr_mean_list = pd.concat([e, f, g], axis=1, ignore_index=True)
    gyr_mean_list.columns = ["gyrx_mean", "gyry_mean", "gyrz_mean"]
#     gyr_mean_list = pd.concat([e, f, g, h], axis=1, ignore_index=True)
#     gyr_mean_list.columns = ["gyrx_mean", "gyry_mean", "gyrz_mean", "gyrn_mean"]
    mean_list = pd.concat([acc_mean_list, gyr_mean_list], axis=1, ignore_index=False)
    return mean_list

data1_d_50_record_mean = acc_gyr_mean_order(data1_d_record_1, data1_d_label_50)
data1_d_60_record_mean = acc_gyr_mean_order(data1_d_record_2, data1_d_label_60)
data1_d_70_record_mean = acc_gyr_mean_order(data1_d_record_3, data1_d_label_70)
data1_d_80_record_mean = acc_gyr_mean_order(data1_d_record_4, data1_d_label_80)

# data2_d_50_record_mean = acc_gyr_mean_order(data2_d_record_1, data2_d_label_50)
# data2_d_60_record_mean = acc_gyr_mean_order(data2_d_record_2, data2_d_label_60)
# data2_d_70_record_mean = acc_gyr_mean_order(data2_d_record_3, data2_d_label_70)
# data2_d_80_record_mean = acc_gyr_mean_order(data2_d_record_4, data2_d_label_80)

# a = data1_d_50_record_mean
# b = data1_d_60_record_mean
# c = data1_d_70_record_mean
# d = data1_d_80_record_mean

# e = data2_d_50_record_mean
# f = data2_d_60_record_mean
# g = data2_d_70_record_mean
# h = data2_d_80_record_mean

# mean_list = pd.concat([a, b, c, d], ignore_index=True)
# mean_list

In [45]:
#加速度x,y,z,norm分散値
def acc_gyr_var_order(data_record, data_label):
    a = [np.var(data_record[i]["加速度X"]) for i in range(0, len(data_label), 1)]
    b = [np.var(data_record[i]["加速度Y"]) for i in range(0, len(data_label), 1)]
    c = [np.var(data_record[i]["加速度Z"]) for i in range(0, len(data_label), 1)]
    d = [np.var(data_record[i]["合成加速度"]) for i in range(0, len(data_label), 1)]
    a = pd.DataFrame(a)
    b = pd.DataFrame(b)
    c = pd.DataFrame(c)
    d = pd.DataFrame(d)
    acc_var_list = pd.concat([a, b, c], axis=1, ignore_index=True)
    acc_var_list.columns = ["accx_var", "accy_var", "accz_var"]
#     acc_var_list = pd.concat([a, b, c, d], axis=1, ignore_index=True)
#     acc_var_list.columns = ["accx_var", "accy_var", "accz_var", "accn_var"]
    e = [np.var(data_record[i]["角速度X"]) for i in range(0, len(data_label), 1)]
    f = [np.var(data_record[i]["角速度Y"]) for i in range(0, len(data_label), 1)]
    g = [np.var(data_record[i]["角速度Z"]) for i in range(0, len(data_label), 1)]
    h = [np.var(data_record[i]["合成角速度"]) for i in range(0, len(data_label), 1)]
    e = pd.DataFrame(e)
    f = pd.DataFrame(f)
    g = pd.DataFrame(g)
    h = pd.DataFrame(h)
    gyr_var_list = pd.concat([e, f, g], axis=1, ignore_index=True)
    gyr_var_list.columns = ["gyrx_var", "gyry_var", "gyrz_var"]
#     gyr_var_list = pd.concat([e, f, g, h], axis=1, ignore_index=True)
#     gyr_var_list.columns = ["gyrx_var", "gyry_var", "gyrz_var", "gyrn_var"]
    var_list = pd.concat([acc_var_list, gyr_var_list], axis=1, ignore_index=False)
    return var_list

data1_d_50_record_var = acc_gyr_var_order(data1_d_record_1, data1_d_label_50)
data1_d_60_record_var = acc_gyr_var_order(data1_d_record_2, data1_d_label_60)
data1_d_70_record_var = acc_gyr_var_order(data1_d_record_3, data1_d_label_70)
data1_d_80_record_var = acc_gyr_var_order(data1_d_record_4, data1_d_label_80)

# data2_d_50_record_var = acc_gyr_var_order(data2_d_record_1, data2_d_label_50)
# data2_d_60_record_var = acc_gyr_var_order(data2_d_record_2, data2_d_label_60)
# data2_d_70_record_var = acc_gyr_var_order(data2_d_record_3, data2_d_label_70)
# data2_d_80_record_var = acc_gyr_var_order(data2_d_record_4, data2_d_label_80)

# a = data1_d_50_record_var
# b = data1_d_60_record_var
# c = data1_d_70_record_var
# d = data1_d_80_record_var

# e = data2_d_50_record_var
# f = data2_d_60_record_var
# g = data2_d_70_record_var
# h = data2_d_80_record_var

# var_list = pd.concat([a, b, c, d, e, f, g, h], ignore_index=True)
# var_list

In [46]:
#加速度x,y,z,norm分散値
def acc_gyr_max_order(data_record, data_label):
    a = [np.max(data_record[i]["加速度X"]) for i in range(0, len(data_label), 1)]
    b = [np.max(data_record[i]["加速度Y"]) for i in range(0, len(data_label), 1)]
    c = [np.max(data_record[i]["加速度Z"]) for i in range(0, len(data_label), 1)]
    d = [np.max(data_record[i]["合成加速度"]) for i in range(0, len(data_label), 1)]
    a = pd.DataFrame(a)
    b = pd.DataFrame(b)
    c = pd.DataFrame(c)
    d = pd.DataFrame(d)
    acc_max_list = pd.concat([a, b, c], axis=1, ignore_index=True)
    acc_max_list.columns = ["accx_max", "accy_max", "accz_max"]
#     acc_max_list = pd.concat([a, b, c, d], axis=1, ignore_index=True)
#     acc_max_list.columns = ["accx_max", "accy_max", "accz_max", "accn_max"]
    e = [np.max(data_record[i]["角速度X"]) for i in range(0, len(data_label), 1)]
    f = [np.max(data_record[i]["角速度Y"]) for i in range(0, len(data_label), 1)]
    g = [np.max(data_record[i]["角速度Z"]) for i in range(0, len(data_label), 1)]
    h = [np.max(data_record[i]["合成角速度"]) for i in range(0, len(data_label), 1)]
    e = pd.DataFrame(e)
    f = pd.DataFrame(f)
    g = pd.DataFrame(g)
    h = pd.DataFrame(h)
    gyr_max_list = pd.concat([e, f, g], axis=1, ignore_index=True)
    gyr_max_list.columns = ["gyrx_max", "gyry_max", "gyrz_max"]
#     gyr_max_list = pd.concat([e, f, g, h], axis=1, ignore_index=True)
#     gyr_max_list.columns = ["gyrx_max", "gyry_max", "gyrz_max", "gyrn_max"]
    max_list = pd.concat([acc_max_list, gyr_max_list], axis=1, ignore_index=False)
    return max_list

data1_d_50_record_max = acc_gyr_max_order(data1_d_record_1, data1_d_label_50)
data1_d_60_record_max = acc_gyr_max_order(data1_d_record_2, data1_d_label_60)
data1_d_70_record_max = acc_gyr_max_order(data1_d_record_3, data1_d_label_70)
data1_d_80_record_max = acc_gyr_max_order(data1_d_record_4, data1_d_label_80)

# data2_d_50_record_max = acc_gyr_max_order(data2_d_record_1, data2_d_label_50)
# data2_d_50_record_max = acc_gyr_max_order(data2_d_record_2, data2_d_label_60)
# data2_d_50_record_max = acc_gyr_max_order(data2_d_record_3, data2_d_label_70)
# data2_d_50_record_max = acc_gyr_max_order(data2_d_record_4, data2_d_label_80)

# a = data1_d_50_record_max
# b = data1_d_60_record_max
# c = data1_d_70_record_max
# d = data1_d_80_record_max

# e = data2_d_50_record_max
# f = data2_d_60_record_max
# g = data2_d_70_record_max
# h = data2_d_80_record_max

# max_list = pd.concat([a, b, c, d, e, f, g, h], ignore_index=True)
# max_list

In [47]:
#加速度x,y,z,norm分散値
def acc_gyr_min_order(data_record, data_label):
    a = [np.min(data_record[i]["加速度X"]) for i in range(0, len(data_label), 1)]
    b = [np.min(data_record[i]["加速度Y"]) for i in range(0, len(data_label), 1)]
    c = [np.min(data_record[i]["加速度Z"]) for i in range(0, len(data_label), 1)]
    d = [np.min(data_record[i]["合成加速度"]) for i in range(0, len(data_label), 1)]
    a = pd.DataFrame(a)
    b = pd.DataFrame(b)
    c = pd.DataFrame(c)
    d = pd.DataFrame(d)
    acc_min_list = pd.concat([a, b, c], axis=1, ignore_index=True)
    acc_min_list.columns = ["accx_min", "accy_min", "accz_min"]
#     acc_min_list = pd.concat([a, b, c, d], axis=1, ignore_index=True)
#     acc_min_list.columns = ["accx_min", "accy_min", "accz_min", "accn_min"]
    e = [np.min(data_record[i]["角速度X"]) for i in range(0, len(data_label), 1)]
    f = [np.min(data_record[i]["角速度Y"]) for i in range(0, len(data_label), 1)]
    g = [np.min(data_record[i]["角速度Z"]) for i in range(0, len(data_label), 1)]
    h = [np.min(data_record[i]["合成角速度"]) for i in range(0, len(data_label), 1)]
    e = pd.DataFrame(e)
    f = pd.DataFrame(f)
    g = pd.DataFrame(g)
    h = pd.DataFrame(h)
    gyr_min_list = pd.concat([e, f, g], axis=1, ignore_index=True)
    gyr_min_list.columns = ["gyrx_min", "gyry_min", "gyrz_min"]
#     gyr_min_list = pd.concat([e, f, g, h], axis=1, ignore_index=True)
#     gyr_min_list.columns = ["gyrx_min", "gyry_min", "gyrz_min", "gyrn_min"]
    min_list = pd.concat([acc_min_list, gyr_min_list], axis=1, ignore_index=False)
    return min_list

data1_d_50_record_min = acc_gyr_min_order(data1_d_record_1, data1_d_label_50)
data1_d_60_record_min = acc_gyr_min_order(data1_d_record_2, data1_d_label_60)
data1_d_70_record_min = acc_gyr_min_order(data1_d_record_3, data1_d_label_70)
data1_d_80_record_min = acc_gyr_min_order(data1_d_record_4, data1_d_label_80)

# data2_d_50_record_min = acc_gyr_min_order(data2_d_record_1, data2_d_label_50)
# data2_d_60_record_min = acc_gyr_min_order(data2_b_record_2, data2_d_label_60)
# data2_d_70_record_min = acc_gyr_min_order(data2_c_record_3, data2_d_label_70)
# data2_d_80_record_min = acc_gyr_min_order(data2_d_record_4, data2_d_label_80)

# a = data1_d_50_record_min
# b = data1_d_60_record_min
# c = data1_d_70_record_min
# d = data1_d_80_record_min

# e = data2_d_50_record_min
# f = data2_d_60_record_min
# g = data2_d_70_record_min
# h = data2_d_80_record_min

# min_list = pd.concat([a, b, c, d, e, f, g, h], ignore_index=True)
# min_list

## 4.5 最大ピーク周波数及び最大ピーク値

In [48]:
def fq_amp_max_order(data_record, data_label):
    #サンプル数とサンプリング周期及びカットする周波数を設定
    N = 512 # サンプル数
    dt = 0.005 # サンプリング周期(sec)
    fc = 100
    t = np.arange(0, N*dt, dt) # 時間軸
    
    f = [data_record[i]["合成加速度"].head(512) for i in range(0, len(data_label), 1)]

    # 高速フーリエ変換(FFT)
    F = np.fft.fft(f)
    F_abs = np.abs(F) # 複素数を絶対値に変換
    F_abs_amp = F_abs / N * 2 # 振幅をもとの信号に揃える(交流成分2倍)
    F_abs_amp[0] = F_abs_amp[0] / 2 # 振幅をもとの信号に揃える(直流成分非2倍)

    # 周波数軸のデータ作成
    fq = np.linspace(0, 1.0/dt, N) # 周波数軸　linspace(開始,終了,分割数)

    # フィルタリング①（周波数でカット）＊＊＊＊＊＊
    F2 = np.copy(F) # FFT結果コピー
    for i in range(0, len(data_record), 1):
        F2[i][(fq > fc)] = 0
    F2_abs = np.abs(F2) # FFTの複素数結果を絶対値に変換
    F2_abs_amp = F2_abs / N * 2 # 振幅をもとの信号に揃える(交流成分2倍)
    for i in range(0, len(data_record), 1):
        F2_abs_amp[i][0] = F2_abs_amp[i][0] / 2 # 振幅をもとの信号に揃える(直流成分非2倍)
    
    fq_amp = pd.DataFrame(fq)
    amp = [pd.DataFrame(F2_abs_amp[i])for i in range(0, len(data_label), 1)]
    a = [pd.concat([fq_amp, amp[i]], axis=1, ignore_index=True)for i in range(0, len(data_label), 1)]

    a = [pd.DataFrame(a[i]).drop(a[i].index[[0]])for i in range(0, len(data_label), 1)]
    a = [a[i][a[i][1] == np.max(a[i][1])]for i in range(0, len(data_label), 1)]
    data_record_accn_fq_amp_max = pd.concat(a, ignore_index=True)
    data_record_accn_fq_amp_max.columns = ["accn_max_fq", "accn_max_peak"]
    
    f = [data_record[i]["合成角速度"].head(512) for i in range(0, len(data_label), 1)]

    # 高速フーリエ変換(FFT)
    F = np.fft.fft(f)
    F_abs = np.abs(F) # 複素数を絶対値に変換
    F_abs_amp = F_abs / N * 2 # 振幅をもとの信号に揃える(交流成分2倍)
    F_abs_amp[0] = F_abs_amp[0] / 2 # 振幅をもとの信号に揃える(直流成分非2倍)

    # 周波数軸のデータ作成
    fq = np.linspace(0, 1.0/dt, N) # 周波数軸　linspace(開始,終了,分割数)

    # フィルタリング①（周波数でカット）＊＊＊＊＊＊
    F2 = np.copy(F) # FFT結果コピー
    for i in range(0, len(data_record), 1):
        F2[i][(fq > fc)] = 0
    F2_abs = np.abs(F2) # FFTの複素数結果を絶対値に変換
    F2_abs_amp = F2_abs / N * 2 # 振幅をもとの信号に揃える(交流成分2倍)
    for i in range(0, len(data_record), 1):
        F2_abs_amp[i][0] = F2_abs_amp[i][0] / 2 # 振幅をもとの信号に揃える(直流成分非2倍)
    
    fq_amp = pd.DataFrame(fq)
    amp = [pd.DataFrame(F2_abs_amp[i])for i in range(0, len(data_label), 1)]
    a = [pd.concat([fq_amp, amp[i]], axis=1, ignore_index=True)for i in range(0, len(data_label), 1)]

    a = [pd.DataFrame(a[i]).drop(a[i].index[[0]])for i in range(0, len(data_label), 1)]
    a = [a[i][a[i][1] == np.max(a[i][1])]for i in range(0, len(data_label), 1)]
    data_record_gyrn_fq_amp_max = pd.concat(a, ignore_index=True)
    data_record_gyrn_fq_amp_max.columns = ["gyrn_max_fq", "gyrn_max_peak"]
    
    b = pd.concat([data_record_accn_fq_amp_max, data_record_gyrn_fq_amp_max], axis=1, ignore_index=False)
    return b

data1_d_50_record_fq_amp_max = fq_amp_max_order(data1_d_record_1, data1_d_label_50)
data1_d_60_record_fq_amp_max = fq_amp_max_order(data1_d_record_2, data1_d_label_60)
data1_d_70_record_fq_amp_max = fq_amp_max_order(data1_d_record_3, data1_d_label_70)
data1_d_80_record_fq_amp_max = fq_amp_max_order(data1_d_record_4, data1_d_label_80)

# data2_d_50_record_fq_amp_max = fq_amp_max_order(data2_d_record_1, data2_d_label_50)
# data2_d_60_record_fq_amp_max = fq_amp_max_order(data2_d_record_2, data2_d_label_60)
# data2_d_70_record_fq_amp_max = fq_amp_max_order(data2_d_record_3, data2_d_label_70)
# data2_d_80_record_fq_amp_max = fq_amp_max_order(data2_d_record_4, data2_d_label_80)

# a = data1_d_50_record_fq_amp_max
# b = data1_d_60_record_fq_amp_max
# c = data1_d_70_record_fq_amp_max
# d = data1_d_80_record_fq_amp_max

# e = data2_d_50_record_fq_amp_max
# f = data2_d_60_record_fq_amp_max
# g = data2_d_70_record_fq_amp_max
# h = data2_d_80_record_fq_amp_max

# fq_amp_max_list = pd.concat([a, b, c, d, e, f, g, h], ignore_index=True)
# fq_amp_max_list

In [49]:
#全特徴量の行列の導出
data1_d_50_record_all_feature = pd.concat([data1_d_50_record_mean, data1_d_50_record_var, data1_d_50_record_max, data1_d_50_record_min, data1_d_50_record_fq_amp_max], axis=1, ignore_index=False)
data1_d_60_record_all_feature = pd.concat([data1_d_60_record_mean, data1_d_60_record_var, data1_d_60_record_max, data1_d_60_record_min, data1_d_60_record_fq_amp_max], axis=1, ignore_index=False)
data1_d_70_record_all_feature = pd.concat([data1_d_70_record_mean, data1_d_70_record_var, data1_d_70_record_max, data1_d_70_record_min, data1_d_70_record_fq_amp_max], axis=1, ignore_index=False)
data1_d_80_record_all_feature = pd.concat([data1_d_80_record_mean, data1_d_80_record_var, data1_d_80_record_max, data1_d_80_record_min, data1_d_80_record_fq_amp_max], axis=1, ignore_index=False)

# data2_d_50_record_all_feature = pd.concat([data2_d_50_record_mean, data2_d_50_record_var, data2_d_50_record_max, data2_d_50_record_min, data2_d_50_record_fq_amp_max], axis=1, ignore_index=False)
# data2_d_60_record_all_feature = pd.concat([data2_d_60_record_mean, data2_d_60_record_var, data2_d_60_record_max, data2_d_60_record_min, data2_d_60_record_fq_amp_max], axis=1, ignore_index=False)
# data2_d_70_record_all_feature = pd.concat([data2_d_70_record_mean, data2_d_70_record_var, data2_d_70_record_max, data2_d_70_record_min, data2_d_70_record_fq_amp_max], axis=1, ignore_index=False)
# data2_d_80_record_all_feature = pd.concat([data2_d_80_record_mean, data2_d_80_record_var, data2_d_80_record_max, data2_d_80_record_min, data2_d_80_record_fq_amp_max], axis=1, ignore_index=False)

In [50]:
#編集したデータの確認(確認のためセッション1のAさんのデータを表示している)
data1_d_60_record_all_feature

,accx_mean,accy_mean,accz_mean,gyrx_mean,gyry_mean,gyrz_mean,accx_var,accy_var,accz_var,gyrx_var,...,accx_min,accy_min,accz_min,gyrx_min,gyry_min,gyrz_min,accn_max_fq,accn_max_peak,gyrn_max_fq,gyrn_max_peak
0,7713.400,-488.355,-5723.911,-73.285,161.165,-4.408,115533.616000,24922.774975,158298.643079,103167.721775,...,6477,-1014,-7002,-969,-2121,-678,14.481409,83.856668,0.782779,186.583991
1,7760.317,-459.719,-5664.404,-73.081,129.147,-2.594,145601.768511,21691.352039,208150.952784,96301.490439,...,6477,-951,-7002,-969,-2121,-678,5.479452,116.057375,0.391389,289.367042
2,7829.777,-443.414,-5577.741,-68.132,200.965,4.568,176957.941271,23935.976604,257609.427919,103009.172576,...,6477,-951,-7002,-1030,-2121,-678,5.479452,117.897418,0.782779,283.254023
3,7938.081,-434.823,-5447.728,-60.532,147.794,3.960,160270.170439,25472.423671,239383.754016,109163.946976,...,6477,-951,-7002,-1030,-2121,-678,5.870841,108.852958,0.782779,361.540628
4,8040.117,-420.990,-5326.090,-61.167,246.664,0.620,75312.227311,19692.293900,143423.707900,88512.185111,...,7036,-853,-6523,-1030,-1707,-678,7.436399,79.466715,0.391389,329.961274
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
624,7110.880,503.203,-6337.904,-19.086,-82.279,43.748,264588.499600,64247.957791,351409.172784,229793.854604,...,6220,-212,-8044,-1573,-8255,-812,9.784736,66.338997,0.391389,673.082865
625,6913.177,406.586,-6552.461,-75.792,-87.928,31.505,127530.217671,26641.624604,199894.310479,182461.180736,...,6220,-212,-8044,-1573,-8255,-843,6.262231,64.863680,0.391389,733.107554
626,6815.271,434.962,-6653.038,-46.912,109.182,2.632,44140.087559,13068.274556,93896.188556,120902.048256,...,6230,-14,-7898,-1573,-8255,-843,3.913894,108.320582,0.391389,1044.320488
627,6823.858,417.047,-6648.090,-39.289,84.448,-4.885,40135.655836,8796.316791,87993.099900,114465.661479,...,6230,-14,-7898,-1573,-8255,-843,3.913894,118.849182,0.782779,1083.530339


In [51]:
#各特徴量データの保存
data1_d_50_record_all_feature.to_csv('feature/D/data1_d_50_record_all_feature.csv', index=False)
data1_d_60_record_all_feature.to_csv('feature/D/data1_d_60_record_all_feature.csv', index=False)
data1_d_70_record_all_feature.to_csv('feature/D/data1_d_70_record_all_feature.csv', index=False)
data1_d_80_record_all_feature.to_csv('feature/D/data1_d_80_record_all_feature.csv', index=False)

# data2_d_50_record_all_feature.to_csv('feature/D/data2_d_50_record_all_feature.csv', index=False)
# data2_d_60_record_all_feature.to_csv('feature/D/data2_d_60_record_all_feature.csv', index=False)
# data2_d_70_record_all_feature.to_csv('feature/D/data2_d_70_record_all_feature.csv', index=False)
# data2_d_80_record_all_feature.to_csv('feature/D/data2_d_80_record_all_feature.csv', index=False)

In [52]:
data1_d_label_50

,start_time,finish_time,emotion_level
0,2020-11-11 12:41:20,2020-11-11 12:41:24,4
1,2020-11-11 12:41:21,2020-11-11 12:41:25,4
2,2020-11-11 12:41:22,2020-11-11 12:41:26,4
3,2020-11-11 12:41:23,2020-11-11 12:41:27,4
4,2020-11-11 12:41:24,2020-11-11 12:41:28,4
...,...,...,...
636,2020-11-11 12:53:49,2020-11-11 12:53:53,-2
637,2020-11-11 12:53:50,2020-11-11 12:53:54,-2
638,2020-11-11 12:53:51,2020-11-11 12:53:55,-2
639,2020-11-11 12:53:52,2020-11-11 12:53:56,-2


In [53]:
data1_d_label_50_time = data1_d_label_50.drop("emotion_level", axis=1)
data1_d_label_60_time = data1_d_label_60.drop("emotion_level", axis=1)
data1_d_label_70_time = data1_d_label_70.drop("emotion_level", axis=1)
data1_d_label_80_time = data1_d_label_80.drop("emotion_level", axis=1)

# data2_d_label_50_time = data2_d_label_50.drop("emotion_level", axis=1)
# data2_d_label_60_time = data2_d_label_60.drop("emotion_level", axis=1)
# data2_d_label_70_time = data2_d_label_70.drop("emotion_level", axis=1)
# data2_d_label_80_time = data2_d_label_80.drop("emotion_level", axis=1)

In [54]:
data1_d_label_50_time

,start_time,finish_time
0,2020-11-11 12:41:20,2020-11-11 12:41:24
1,2020-11-11 12:41:21,2020-11-11 12:41:25
2,2020-11-11 12:41:22,2020-11-11 12:41:26
3,2020-11-11 12:41:23,2020-11-11 12:41:27
4,2020-11-11 12:41:24,2020-11-11 12:41:28
...,...,...
636,2020-11-11 12:53:49,2020-11-11 12:53:53
637,2020-11-11 12:53:50,2020-11-11 12:53:54
638,2020-11-11 12:53:51,2020-11-11 12:53:55
639,2020-11-11 12:53:52,2020-11-11 12:53:56


In [55]:
data1_d_50_record_all_feature = pd.concat([data1_d_label_50_time, data1_d_50_record_all_feature], axis=1)
data1_d_60_record_all_feature = pd.concat([data1_d_label_60_time, data1_d_60_record_all_feature], axis=1)
data1_d_70_record_all_feature = pd.concat([data1_d_label_70_time, data1_d_70_record_all_feature], axis=1)
data1_d_80_record_all_feature = pd.concat([data1_d_label_80_time, data1_d_80_record_all_feature], axis=1)

# data2_d_50_record_all_feature = pd.concat([data2_d_label_50_time, data2_d_50_record_all_feature], axis=1)
# data2_d_60_record_all_feature = pd.concat([data2_d_label_60_time, data2_d_60_record_all_feature], axis=1)
# data2_d_70_record_all_feature = pd.concat([data2_d_label_70_time, data2_d_70_record_all_feature], axis=1)
# data2_d_80_record_all_feature = pd.concat([data2_d_label_80_time, data2_d_80_record_all_feature], axis=1)

In [56]:
data1_d_60_record_all_feature

,start_time,finish_time,accx_mean,accy_mean,accz_mean,gyrx_mean,gyry_mean,gyrz_mean,accx_var,accy_var,...,accx_min,accy_min,accz_min,gyrx_min,gyry_min,gyrz_min,accn_max_fq,accn_max_peak,gyrn_max_fq,gyrn_max_peak
0,2020-11-11 12:41:21,2020-11-11 12:41:25,7713.400,-488.355,-5723.911,-73.285,161.165,-4.408,115533.616000,24922.774975,...,6477,-1014,-7002,-969,-2121,-678,14.481409,83.856668,0.782779,186.583991
1,2020-11-11 12:41:22,2020-11-11 12:41:26,7760.317,-459.719,-5664.404,-73.081,129.147,-2.594,145601.768511,21691.352039,...,6477,-951,-7002,-969,-2121,-678,5.479452,116.057375,0.391389,289.367042
2,2020-11-11 12:41:23,2020-11-11 12:41:27,7829.777,-443.414,-5577.741,-68.132,200.965,4.568,176957.941271,23935.976604,...,6477,-951,-7002,-1030,-2121,-678,5.479452,117.897418,0.782779,283.254023
3,2020-11-11 12:41:24,2020-11-11 12:41:28,7938.081,-434.823,-5447.728,-60.532,147.794,3.960,160270.170439,25472.423671,...,6477,-951,-7002,-1030,-2121,-678,5.870841,108.852958,0.782779,361.540628
4,2020-11-11 12:41:25,2020-11-11 12:41:29,8040.117,-420.990,-5326.090,-61.167,246.664,0.620,75312.227311,19692.293900,...,7036,-853,-6523,-1030,-1707,-678,7.436399,79.466715,0.391389,329.961274
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
624,2020-11-11 12:53:49,2020-11-11 12:53:53,7110.880,503.203,-6337.904,-19.086,-82.279,43.748,264588.499600,64247.957791,...,6220,-212,-8044,-1573,-8255,-812,9.784736,66.338997,0.391389,673.082865
625,2020-11-11 12:53:50,2020-11-11 12:53:54,6913.177,406.586,-6552.461,-75.792,-87.928,31.505,127530.217671,26641.624604,...,6220,-212,-8044,-1573,-8255,-843,6.262231,64.863680,0.391389,733.107554
626,2020-11-11 12:53:51,2020-11-11 12:53:55,6815.271,434.962,-6653.038,-46.912,109.182,2.632,44140.087559,13068.274556,...,6230,-14,-7898,-1573,-8255,-843,3.913894,108.320582,0.391389,1044.320488
627,2020-11-11 12:53:52,2020-11-11 12:53:56,6823.858,417.047,-6648.090,-39.289,84.448,-4.885,40135.655836,8796.316791,...,6230,-14,-7898,-1573,-8255,-843,3.913894,118.849182,0.782779,1083.530339


In [57]:
data1_d_50_record_all_feature.to_csv('feature/D/data1_d_50_record_all_feature.csv', index=False)
data1_d_60_record_all_feature.to_csv('feature/D/data1_d_60_record_all_feature.csv', index=False)
data1_d_70_record_all_feature.to_csv('feature/D/data1_d_70_record_all_feature.csv', index=False)
data1_d_80_record_all_feature.to_csv('feature/D/data1_d_80_record_all_feature.csv', index=False)

# data2_d_50_record_all_feature.to_csv('feature/D/data2_d_50_record_all_feature.csv', index=False)
# data2_d_60_record_all_feature.to_csv('feature/D/data2_d_60_record_all_feature.csv', index=False)
# data2_d_70_record_all_feature.to_csv('feature/D/data2_d_70_record_all_feature.csv', index=False)
# data2_d_80_record_all_feature.to_csv('feature/D/data2_d_80_record_all_feature.csv', index=False)

In [58]:
data1_d_60_record_all_feature

,start_time,finish_time,accx_mean,accy_mean,accz_mean,gyrx_mean,gyry_mean,gyrz_mean,accx_var,accy_var,...,accx_min,accy_min,accz_min,gyrx_min,gyry_min,gyrz_min,accn_max_fq,accn_max_peak,gyrn_max_fq,gyrn_max_peak
0,2020-11-11 12:41:21,2020-11-11 12:41:25,7713.400,-488.355,-5723.911,-73.285,161.165,-4.408,115533.616000,24922.774975,...,6477,-1014,-7002,-969,-2121,-678,14.481409,83.856668,0.782779,186.583991
1,2020-11-11 12:41:22,2020-11-11 12:41:26,7760.317,-459.719,-5664.404,-73.081,129.147,-2.594,145601.768511,21691.352039,...,6477,-951,-7002,-969,-2121,-678,5.479452,116.057375,0.391389,289.367042
2,2020-11-11 12:41:23,2020-11-11 12:41:27,7829.777,-443.414,-5577.741,-68.132,200.965,4.568,176957.941271,23935.976604,...,6477,-951,-7002,-1030,-2121,-678,5.479452,117.897418,0.782779,283.254023
3,2020-11-11 12:41:24,2020-11-11 12:41:28,7938.081,-434.823,-5447.728,-60.532,147.794,3.960,160270.170439,25472.423671,...,6477,-951,-7002,-1030,-2121,-678,5.870841,108.852958,0.782779,361.540628
4,2020-11-11 12:41:25,2020-11-11 12:41:29,8040.117,-420.990,-5326.090,-61.167,246.664,0.620,75312.227311,19692.293900,...,7036,-853,-6523,-1030,-1707,-678,7.436399,79.466715,0.391389,329.961274
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
624,2020-11-11 12:53:49,2020-11-11 12:53:53,7110.880,503.203,-6337.904,-19.086,-82.279,43.748,264588.499600,64247.957791,...,6220,-212,-8044,-1573,-8255,-812,9.784736,66.338997,0.391389,673.082865
625,2020-11-11 12:53:50,2020-11-11 12:53:54,6913.177,406.586,-6552.461,-75.792,-87.928,31.505,127530.217671,26641.624604,...,6220,-212,-8044,-1573,-8255,-843,6.262231,64.863680,0.391389,733.107554
626,2020-11-11 12:53:51,2020-11-11 12:53:55,6815.271,434.962,-6653.038,-46.912,109.182,2.632,44140.087559,13068.274556,...,6230,-14,-7898,-1573,-8255,-843,3.913894,108.320582,0.391389,1044.320488
627,2020-11-11 12:53:52,2020-11-11 12:53:56,6823.858,417.047,-6648.090,-39.289,84.448,-4.885,40135.655836,8796.316791,...,6230,-14,-7898,-1573,-8255,-843,3.913894,118.849182,0.782779,1083.530339
